In [1]:
import pandas as pd 
import numpy as np 
import os 
import tomotopy as tp
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from langdetect import detect 
import time
import plotly.express as px
from datetime import datetime

nltk.download('stopwords')
stop_words = stopwords.words('english')
porter = PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def preprocessing_text(indx, text): 
    if indx%10000==0:
        print(indx)
    text = text.lower()
    text.replace('@','')
    text_p = "".join([char for char in text if char not in string.punctuation])
    words = text_p.strip().split()
    filtered_words = [word for word in words if word not in stop_words]
    filtered_words = [word for word in filtered_words if len(word) >2]
    filtered_words = [word for word in filtered_words if word[0:4] !='http']
    stemmed = [porter.stem(word) for word in filtered_words]
   # stemmed = filtered_words
    try:
        stemmed.remove('rt')
    except:
        pass
    return stemmed


def twitter_datetime_preprocess(dt_obj):
    
    d = datetime.strptime(dt_obj,'%a %b %d %H:%M:%S %z %Y')
    d = datetime.strftime(d,"%Y/%m")
    return d 

In [3]:
# reset directory
os.chdir("C:/Users/user/Documents/LSHTM_Social_Media/")

# set input files directory
geo_dir = 'data/twitter_prediction/data/loc/output/'
pred_dir = 'data/twitter_prediction/data/output/'
combined_dir ='data/twitter_prediction/data/geotagged_pred/'
geo_files = os.listdir(geo_dir)
pred_files = os.listdir(pred_dir)

In [4]:
# for tidying messy tweets

def messy_tweet_id(input_id): 
    try:
        input_id = str(int(float(input_id)))
    except:
        pass
    return input_id

# Gross Descriptive

In [5]:
# read in files
relevance_prop_agg = {}
for indx, file in enumerate(pred_files):
    pred_data = pd.read_csv(pred_dir+file)
    relevance_prop = len(pred_data[pred_data['pred_relevance']==1])/len(pred_data)
    relevance_prop_agg[file[0:8]] = relevance_prop
    print(file)


# clean data 
relevance_prop_agg = pd.DataFrame({'date':list(relevance_prop_agg.keys()),'prop': list(relevance_prop_agg.values())})
relevance_prop_agg['date'] = relevance_prop_agg['date'].map(lambda x: datetime.strptime(x,'%Y%m%d'))

# plot proportion of relevant twitter posts
fig = px.line(relevance_prop_agg, x='date', y='prop', template = 'simple_white')
fig.update_traces(line_color='#12222F')
fig.update_layout(template='simple_white', title= dict(text='Proportion of Twitter Posts Being Relevant to COVID Vaccines'))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Count')
fig.show()

# plot number of twitter posts collected 
line_df = pd.read_csv("data/twittter_file_count.csv")
line_df['file'] = line_df['file'].map(lambda x: x.split('-')[2][0:8])
line_df['file'] = line_df['file'].map(lambda x:datetime.strptime(x,'%Y%m%d'))
line_df.drop('Unnamed: 0',1, inplace=True)
line_df.columns = ['Date', 'Count']

fig = px.line(line_df, x='Date', y='Count', template = 'simple_white')
fig.update_traces(line_color='#12222F')
fig.update_layout(template='simple_white', title= dict(text='Total Number of Twitter Posts Collected'))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Count')
fig.show()

20200914_pred.csv
20200915_pred.csv
20200916_pred.csv
20200917_pred.csv
20200918_pred.csv
20200919_pred.csv
20200920_pred.csv


KeyboardInterrupt: 

In [14]:
# combine geotag data with tweets

for indx, file in enumerate(pred_files):
    pred_data = pd.read_csv(pred_dir+file)
    geo_data = pd.read_csv(geo_dir+ f'{indx}_geo.csv')
    pred_data = pred_data.drop('Unnamed: 0',1)
    geo_data = geo_data.drop('Unnamed: 0',1)
    geo_data['tweet_id'] = geo_data['tweet_id'].map(messy_tweet_id)
    geo_data.drop_duplicates(inplace=True)
    
    combined_data = pd.merge(geo_data, pred_data, how='left', on='tweet_id')
    combined_data = combined_data.dropna(subset=['dt'])
    
    relevant_num = len(combined_data[combined_data['pred_relevance']==1])
    
    combined_data.to_csv(combined_dir+f'{file[0:8]}_geotagged_pred.csv')
    print(file)
    print(relevant_num)

20200914_pred.csv
5058
20200915_pred.csv
2914
20200916_pred.csv
2635


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



20200917_pred.csv
3453


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [6]:
# read in geotagging information

combined_files  = os.listdir(combined_dir)

combined_relevant = pd.DataFrame()
for file in combined_files: 
    df = pd.read_csv(combined_dir+file)
    df = df[df['pred_relevance']==1]
    combined_relevant = combined_relevant.append(df)
    print(file)
    
combined_relevant = combined_relevant.drop('Unnamed: 0',1)
    

20200914_geotagged_pred.csv
20200915_geotagged_pred.csv
20200916_geotagged_pred.csv
20200917_geotagged_pred.csv
20200918_geotagged_pred.csv
20200919_geotagged_pred.csv
20200920_geotagged_pred.csv
20200921_geotagged_pred.csv
20200922_geotagged_pred.csv
20200923_geotagged_pred.csv
20200924_geotagged_pred.csv
20200925_geotagged_pred.csv
20200926_geotagged_pred.csv
20200927_geotagged_pred.csv
20200928_geotagged_pred.csv
20200929_geotagged_pred.csv
20200930_geotagged_pred.csv
20201001_geotagged_pred.csv
20201002_geotagged_pred.csv
20201003_geotagged_pred.csv
20201004_geotagged_pred.csv


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


20201005_geotagged_pred.csv
20201006_geotagged_pred.csv
20201007_geotagged_pred.csv
20201008_geotagged_pred.csv


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


20201009_geotagged_pred.csv
20201010_geotagged_pred.csv
20201011_geotagged_pred.csv
20201012_geotagged_pred.csv


...

20210527_geotagged_pred.csv


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


20210528_geotagged_pred.csv


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


20210529_geotagged_pred.csv
20210530_geotagged_pred.csv
20210531_geotagged_pred.csv


In [8]:
# count top locations 
loc_count  =  combined_relevant.groupby('tweet_loc').count()
loc_count = loc_count.reset_index()
loc_count = loc_count[['tweet_loc','UID']]
loc_count.sort_values(by='UID', ascending = False).head(20)



,tweet_loc,UID
490,France,541935
45,Argentina,306296
1580,Thailand,214585
1673,United States,192341
192,Brazil,152826
251,Canada,107101
342,Colombia,94478
1223,Peru,91982
717,India,81543
305,Chile,78051


In [9]:
combined_relevant['dt']=combined_relevant['dt'].map(twitter_datetime_preprocess)
target_loc = list(loc_count.sort_values(by='UID', ascending = False).head(10)['tweet_loc'])
filtered_combined_relevant = combined_relevant[combined_relevant['tweet_loc'].isin(target_loc)]

# Sentiment By Location

In [108]:
by_loc_sentiment = filtered_combined_relevant[['screen_name', 'tweet_loc','dt', 'antivax_pred']]

In [109]:
by_loc_sentiment['antivax_pred']=by_loc_sentiment['antivax_pred'].map(int)

<ipython-input-109-e986db76da04>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [110]:
by_loc_sentiment = by_loc_sentiment.groupby(['dt','tweet_loc','screen_name'])['antivax_pred'].mean()

In [111]:
by_loc_sentiment = by_loc_sentiment.reset_index()

In [112]:
dt_period = ['2020/09','2020/10','2020/11','2020/12','2021/01','2021/02','2021/03','2021/04','2021/05']
by_loc_sentiment = by_loc_sentiment[by_loc_sentiment['dt'].isin(dt_period)]

In [113]:
by_loc_sentiment = by_loc_sentiment.groupby(['dt','tweet_loc'])['antivax_pred'].sum()/by_loc_sentiment.groupby(['dt','tweet_loc'])['antivax_pred'].count()

In [114]:
by_loc_sentiment = by_loc_sentiment.reset_index()

In [115]:
SA_ls = ['Argentina','Brazil','Chile','Colombia','Peru']
NA_EU_ls = ['Canada','United States','France']
asia_ls = ['India', 'Thailand']

In [116]:
set(by_loc_sentiment['tweet_loc'])

{'Argentina',
 'Brazil',
 'Canada',
 'Chile',
 'Colombia',
 'France',
 'India',
 'Peru',
 'Thailand',
 'United States'}

In [117]:
SA = by_loc_sentiment[by_loc_sentiment['tweet_loc'].isin(SA_ls)]
px.line(SA, x= 'dt', y = 'antivax_pred', color='tweet_loc')

# Gross Topic Distribution

In [13]:
location_count  =  filtered_combined_relevant[['tweet_loc','UID','dt']]
location_count = location_count.groupby(['tweet_loc','dt']).count()
data_dir = 'data/twitter_prediction/data/topic_pred/'
topic_files = os.listdir(data_dir)
date_ls = [file[0:6] for file in topic_files]
date_ls = list(set(date_ls))
date_ls = sorted(date_ls)

topic_decoded = pd.read_csv('data/twitter_prediction/data/other_data/twitter_topics_aug4.csv')


In [14]:
topic_decoded_dict= {}
for indx in range(len(topic_decoded)):
    topic_decoded_dict[topic_decoded.iloc[indx]['number']]= topic_decoded.iloc[indx]['group']

In [ ]:
# Topic Distribution by location

geo_ls = []
geo_linked_df = pd.DataFrame()
for indx, file in enumerate(topic_files):
    #df = pd.read_csv(data_dir+target_file)
    geo_data = pd.read_csv(geo_dir+ f'{indx}_geo.csv')
    geo_data['tweet_id'] = geo_data['tweet_id'].map(messy_tweet_id)
    geo_data.drop_duplicates(inplace=True)
    geo_data = geo_data[['tweet_id', 'tweet_loc']]
    geo_linked_df = geo_linked_df.append(geo_data)
    geo_ls.append(list(geo_data['tweet_id']))
    print(file)

geo_ls = [_id for subls in geo_ls for _id in subls]

geo_tagged_df = pd.DataFrame()
for indx, file in enumerate(topic_files):
    df = pd.read_csv(data_dir+file)
    df['tweet_id'] = df['tweet_id'].map(messy_tweet_id)
    df = df[df['tweet_id'].isin(geo_ls)]
    geo_tagged_df = geo_tagged_df.append(df)
    print(file)

geo_location = geo_linked_df.groupby('tweet_loc').count()

locations = list(geo_location.sort_values(by='tweet_id', ascending =False).head(10).index)

locations.append('Chile')

loc_topic_distribution_agg = pd.DataFrame()
loc_hesitant_agg = pd.DataFrame()

for indx, loc in enumerate(locations):
    target_df = geo_linked_df[geo_linked_df['tweet_loc']==loc]
    target_df =pd.merge(target_df,geo_tagged_df, on='tweet_id', how='left')
    target_df.dropna(subset=['dt'], inplace=True)
    target_df['topic'] = target_df['topic'].map(topic_decoded_dict)
    loc_stat = target_df.groupby('topic')['text'].count()
    loc_hesitant = target_df[target_df['antivax_pred']==1].groupby(['topic'])['antivax_pred'].count()/target_df.groupby(['topic'])['antivax_pred'].count()
    if indx==0:
        loc_topic_distribution_agg = loc_stat
        loc_hesitant_agg=loc_hesitant
    else:
        loc_topic_distribution_agg = pd.merge(loc_topic_distribution_agg, loc_stat, left_index=True, right_index=True)
        loc_hesitant_agg = pd.merge(loc_hesitant_agg, loc_hesitant, left_index=True, right_index=True)
    print(loc)
    

loc_topic_distribution_agg.columns = locations
loc_topic_distribution_agg.reset_index(inplace=True)

loc_topic_distribution_agg=loc_topic_distribution_agg[loc_topic_distribution_agg['topic']!='IR']
loc_topic_distribution_agg=loc_topic_distribution_agg[loc_topic_distribution_agg['topic']!='Vaccine General']

loc_topic_distribution_agg

import plotly.express as px

for loc in locations:
        fig = px.pie(loc_topic_distribution_agg, values=f'{loc}', names='topic',color='topic',
                     title=f'Topic Distribution for {loc}')
        #fig.update_trace(sort=False)
        fig.write_html(f'topic_pie_{loc}.html')

loc_hesitant_agg = loc_hesitant_agg*100
loc_hesitant_agg.columns = locations
loc_hesitant_agg.reset_index(inplace=True)
loc_hesitant_agg=loc_hesitant_agg[loc_hesitant_agg['topic']!='IR']
loc_hesitant_agg=loc_hesitant_agg[loc_hesitant_agg['topic']!='Vaccine General']




for loc in locations:
        fig = px.bar(loc_hesitant_agg, y=f'{loc}', x='topic',color='topic',
                     title=f'Proportion of Hesitant Posts for {loc}')
        fig.update_layout(template='simple_white')
        fig.update_xaxes(title_text = 'Topic')
        fig.update_yaxes(title_text = 'Percentage of Hesitant Posts')

        #fig.update_trace(sort=False)
        fig.write_html(f'topic_bar_{loc}.html')

In [15]:
combined_stat=pd.DataFrame()

for date in date_ls:
    month_agg = pd.DataFrame()
    target_file_ls = [file for file in topic_files if file[0:6]==date]
    for target_file in target_file_ls: 
        df = pd.read_csv(data_dir+target_file)
        month_agg= month_agg.append(df)

    month_agg['topic'] = month_agg['topic'].map(topic_decoded_dict)
    month_agg['antivax_pred'] = month_agg['antivax_pred'].map({1:-1,0:1})
    month_agg['tweet_sen'] = month_agg['tweet_sen'].map({0:-1,1:1})

    month_stats_count = month_agg.groupby(['topic'])['text'].count()
    vac_sen_stats = month_agg.groupby(['topic'])['antivax_pred'].mean()
    tweet_sen_stats =  month_agg.groupby(['topic'])['tweet_sen'].mean()
    
    combined_month_stat = pd.merge(month_stats_count, vac_sen_stats, left_index=True, right_index=True)
    combined_month_stat = pd.merge(combined_month_stat, tweet_sen_stats, left_index=True, right_index=True)
    combined_month_stat['month']=date
    
    combined_stat = combined_stat.append(combined_month_stat)
    
    print(date)

202009
202010
202011
202012
202101
202102
202103
202104
202105


In [20]:
combined_stat.columns = ['topic','count','vac_sen','topic_sen', 'month']
combined_stat.reset_index(inplace=True)
combined_stat = combined_stat[combined_stat['topic']!='IR']
combined_stat = combined_stat[combined_stat['topic']!='Vaccine General']

In [21]:
fig = px.line(combined_stat, x='month', y='count', color= 'topic')
fig.update_layout(template='simple_white', title= dict(text='Twitter Posts By Topic'))
fig.update_xaxes(title_text = 'Month')
fig.update_yaxes(title_text = 'Count')

In [23]:
daily_stat_agg = pd.DataFrame()
for date in date_ls:
    
    target_file_ls = [file for file in topic_files if file[0:6]==date]
    for target_file in target_file_ls: 
        df = pd.read_csv(data_dir+target_file)
        df['topic'] = df['topic'].map(topic_decoded_dict)
        
        df_length = df.groupby(['topic'])['antivax_pred'].count()
        df_neg = df[df['antivax_pred']==1].groupby(['topic'])['antivax_pred'].count()
        
        day_stat = pd.merge(df_neg,df_length, left_index=True, right_index=True)
        day_stat.columns =['neg','gross']
        day_stat['date'] = target_file[0:8]
        daily_stat_agg = daily_stat_agg.append(day_stat)
        
    print(date)

202009
202010
202011
202012
202101
202102
202103
202104
202105


In [24]:
daily_stat_agg['prop'] = daily_stat_agg['neg']/daily_stat_agg['gross']
daily_stat_agg.reset_index(inplace=True)
daily_stat_agg= daily_stat_agg[daily_stat_agg['topic']!='IR']
top_var= list(set(daily_stat_agg['topic']))

In [26]:
# plot distribution by topic

for target_var in top_var:
    ploting_df = daily_stat_agg[daily_stat_agg['topic']==target_var]
    ploting_df['net_MA7'] = ploting_df['prop'].rolling(7).mean()
    ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strptime(x, '%Y%m%d'))

    # plot
    fig_scatter = px.scatter(ploting_df, x='date', y = 'prop', template ='simple_white')
    fig_scatter.update_traces(marker=dict(color='grey', size=4,opacity=0.5))
    fig_line = px.line(ploting_df, x='date', y = 'net_MA7', template ='simple_white')
    fig_line.update_traces(line_color='#12222F')
    fig_scatter.add_traces(fig_line._data)

    fig_scatter.update_layout(template='simple_white', title= dict(text=f'Twitter Vaccine Sentiment Parity ({target_var})'))
    fig_scatter.update_xaxes(title_text = 'Month')
    fig_scatter.update_yaxes(title_text = 'Proportion of Hesitant Posts')
    fig_scatter.write_html(f'plot0_{target_var}.html')

<ipython-input-26-b1b621caa77d>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-b1b621caa77d>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
# plot trends by topic

for target_var in top_var:
        print(target_var)
        ploting_df = daily_stat_agg[daily_stat_agg['topic']==target_var]

        ploting_df['prop'] = ploting_df['prop']*100
        # ploting_df['net_MA7'] = ploting_df['prop'].rolling(7).mean()
        ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strptime(x, '%Y%m%d'))
        ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strftime(x, '%Y-%m'))

Vaccine Delivery
Vaccine General
Vaccine Safety
Vaccine Effectiveness
Vaccine Trial
Vaccine Incentives


<ipython-input-27-81d25aa27c62>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-27-81d25aa27c62>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-27-81d25aa27c62>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
# test for trend 

for target_var in top_var:
    print(target_var)
    ploting_df = daily_stat_agg[daily_stat_agg['topic']==target_var]

    ploting_df['prop'] = ploting_df['prop']*100
    # ploting_df['net_MA7'] = ploting_df['prop'].rolling(7).mean()
    ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strptime(x, '%Y%m%d'))
    ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strftime(x, '%Y-%m'))
    mos_df = ploting_df.groupby('date')['prop'].mean()
    mos_df = mos_df.reset_index()
    mos_df['mos'] = range(len(mos_df))


    import statsmodels.api as sm
    #define response variable
    y = mos_df['prop']
    #define explanatory variable
    x = mos_df[['mos']]
    #add constant to predictor variables
    x = sm.add_constant(x)
    #fit linear regression model
    model = sm.OLS(y, x).fit()
    #view model summary
    print(model.summary())

Vaccine Delivery
                            OLS Regression Results                            
Dep. Variable:                   prop   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     45.57
Date:                Tue, 24 Aug 2021   Prob (F-statistic):           0.000265
Time:                        11:55:53   Log-Likelihood:                -8.1398
No. Observations:                   9   AIC:                             20.28
Df Residuals:                       7   BIC:                             20.67
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.9276      0.417   

<ipython-input-30-f192b64018b4>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-f192b64018b4>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-f192b64018b4>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\anaconda3\

In [29]:
var_df =pd.DataFrame()

for target_var in top_var:
    print(target_var)
    ploting_df = daily_stat_agg[daily_stat_agg['topic']==target_var]

    ploting_df['prop'] = ploting_df['prop']*100
    # ploting_df['net_MA7'] = ploting_df['prop'].rolling(7).mean()
    ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strptime(x, '%Y%m%d'))
    ploting_df['date'] = ploting_df['date'].map(lambda x:datetime.strftime(x, '%Y-%m'))
    mos_df = ploting_df.groupby('date')['prop'].mean()
    var_df = pd.concat([var_df, mos_df], axis=1)

Vaccine Delivery
Vaccine General
Vaccine Safety
Vaccine Effectiveness
Vaccine Trial
Vaccine Incentives


<ipython-input-29-79dc565dd3ec>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-29-79dc565dd3ec>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-29-79dc565dd3ec>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
var_df.columns = top_var
var_df = var_df.transpose()
var_df = round(var_df,2)
var_df.to_csv('twitter_sentiment_df.csv')

# Topic Interaction Pattern

In [33]:
df_stat = pd.DataFrame()
for date in date_ls:
    
    target_file_ls = [file for file in topic_files if file[0:6]==date]
    for target_file in target_file_ls: 
        df = pd.read_csv(data_dir+target_file)
        df['topic'] = df['topic'].map(topic_decoded_dict)
        df = df[df['mode']!='prim']
        df = df[['dt','quote_num', 'reply_num', 'retweet_num', 'fav_num','antivax_pred', 'topic']]
        df_stat = df_stat.append(df)
        
    print(date)

202009
202010
202011
202012
202101
202102
202103
202104
202105


In [34]:
df_stat = df_stat[df_stat['dt'].isna()==False]
df_stat['dt'] = df_stat['dt'].map(lambda x:datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'))

In [35]:
df_stat['active_interaction']= df_stat['quote_num']+df_stat['reply_num']+df_stat['retweet_num']

In [36]:
df_stat['antivax_pred']=df_stat['antivax_pred'].map({0:'Non-hesitant', 1:'hesitant'})

In [37]:
var_ls = list(set(df_stat['topic']))
var_ls.remove('IR')
var_ls.remove('Vaccine General')

In [38]:
for topic_var in var_ls: 
    plot_df = df_stat[df_stat['topic']==topic_var]

    plot_df.columns = ['dt', 'quote_num', 'reply_num', 'retweet_num', 'fav_num',
       'Attitude', 'topic', 'active_interaction']
    
    fig = px.scatter(plot_df, x='active_interaction', y='fav_num', log_x=True,log_y=True, 
                    template ='simple_white', 
                     title = f'Twitter Interaction Statistics for {topic_var}')
    fig.update_traces(marker = dict(opacity=0.01))
    fig.update_xaxes(title_text = 'Active Interactions')
    fig.update_yaxes(title_text = 'Passive Interactions')

    fig.write_html(f'interaction_stats_{topic_var}.html')

# Topic Distribution by location

In [62]:
geo_ls = []
geo_linked_df = pd.DataFrame()
for indx, file in enumerate(topic_files):
    #df = pd.read_csv(data_dir+target_file)
    geo_data = pd.read_csv(geo_dir+ f'{indx}_geo.csv')
    geo_data['tweet_id'] = geo_data['tweet_id'].map(messy_tweet_id)
    geo_data.drop_duplicates(inplace=True)
    geo_data = geo_data[['tweet_id', 'tweet_loc']]
    geo_linked_df = geo_linked_df.append(geo_data)
    geo_ls.append(list(geo_data['tweet_id']))
    print(file)

20200914.csv
20200915.csv
20200916.csv


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



20200917.csv
20200918.csv
20200919.csv
20200920.csv
20200921.csv
20200922.csv
20200923.csv
20200924.csv
20200925.csv
20200926.csv
20200927.csv
20200928.csv
20200929.csv
20200930.csv
20201001.csv
20201002.csv
20201003.csv
20201004.csv
20201005.csv
20201006.csv
20201007.csv
20201008.csv
20201009.csv
20201010.csv
20201011.csv
20201012.csv
20201013.csv
20201014.csv
20201015.csv
20201016.csv
20201017.csv
20201018.csv
20201019.csv
20201020.csv
20201021.csv
20201022.csv
20201023.csv
20201024.csv
20201025.csv
20201026.csv
20201027.csv
20201028.csv
20201029.csv
20201030.csv
20201031.csv
20201101.csv
20201102.csv
20201103.csv
20201104.csv
20201105.csv
20201106.csv
20201107.csv
20201108.csv
20201109.csv
20201110.csv
20201111.csv
20201112.csv
20201113.csv
20201114.csv
20201115.csv
20201116.csv
20201117.csv
20201118.csv
20201119.csv
20201120.csv
20201121.csv
20201122.csv
20201123.csv
20201124.csv
20201125.csv
20201126.csv
20201127.csv
20201128.csv
20201129.csv
20201130.csv
20201201.csv
20201202.csv

In [63]:
geo_ls = [_id for subls in geo_ls for _id in subls]

In [64]:
geo_tagged_df = pd.DataFrame()
for indx, file in enumerate(topic_files):
    df = pd.read_csv(data_dir+file)
    df['tweet_id'] = df['tweet_id'].map(messy_tweet_id)
    df = df[df['tweet_id'].isin(geo_ls)]
    geo_tagged_df = geo_tagged_df.append(df)
    print(file)

20200914.csv
20200915.csv
20200916.csv
20200917.csv
20200918.csv
20200919.csv
20200920.csv
20200921.csv
20200922.csv
20200923.csv
20200924.csv
20200925.csv
20200926.csv
20200927.csv
20200928.csv
20200929.csv
20200930.csv
20201001.csv
20201002.csv
20201003.csv
20201004.csv
20201005.csv
20201006.csv
20201007.csv
20201008.csv
20201009.csv
20201010.csv
20201011.csv
20201012.csv
20201013.csv
20201014.csv
20201015.csv
20201016.csv
20201017.csv
20201018.csv
20201019.csv
20201020.csv
20201021.csv
20201022.csv
20201023.csv
20201024.csv
20201025.csv
20201026.csv
20201027.csv
20201028.csv
20201029.csv
20201030.csv
20201031.csv
20201101.csv
20201102.csv
20201103.csv
20201104.csv
20201105.csv
20201106.csv
20201107.csv
20201108.csv
20201109.csv
20201110.csv
20201111.csv
20201112.csv
20201113.csv
20201114.csv
20201115.csv
20201116.csv
20201117.csv
20201118.csv
20201119.csv
20201120.csv
20201121.csv
20201122.csv
20201123.csv
20201124.csv
20201125.csv
20201126.csv
20201127.csv
20201128.csv
20201129.csv

In [65]:
geo_location = geo_linked_df.groupby('tweet_loc').count()

In [66]:
locations = list(geo_location.sort_values(by='tweet_id', ascending =False).head(10).index)

In [78]:
locations.append('Chile')

In [79]:
loc_topic_distribution_agg = pd.DataFrame()
loc_hesitant_agg = pd.DataFrame()

for indx, loc in enumerate(locations):
    target_df = geo_linked_df[geo_linked_df['tweet_loc']==loc]
    target_df =pd.merge(target_df,geo_tagged_df, on='tweet_id', how='left')
    target_df.dropna(subset=['dt'], inplace=True)
    target_df['topic'] = target_df['topic'].map(topic_decoded_dict)
    loc_stat = target_df.groupby('topic')['text'].count()
    loc_hesitant = target_df[target_df['antivax_pred']==1].groupby(['topic'])['antivax_pred'].count()/target_df.groupby(['topic'])['antivax_pred'].count()
    if indx==0:
        loc_topic_distribution_agg = loc_stat
        loc_hesitant_agg=loc_hesitant
    else:
        loc_topic_distribution_agg = pd.merge(loc_topic_distribution_agg, loc_stat, left_index=True, right_index=True)
        loc_hesitant_agg = pd.merge(loc_hesitant_agg, loc_hesitant, left_index=True, right_index=True)
    print(loc)
    

Indonesia
United States
France
Argentina
Thailand
Brazil
Canada
India
Colombia
Peru
Chile


In [80]:
loc_topic_distribution_agg.columns = locations
loc_topic_distribution_agg.reset_index(inplace=True)

In [81]:
loc_topic_distribution_agg=loc_topic_distribution_agg[loc_topic_distribution_agg['topic']!='IR']
loc_topic_distribution_agg=loc_topic_distribution_agg[loc_topic_distribution_agg['topic']!='Vaccine General']

In [82]:
loc_topic_distribution_agg

,topic,Indonesia,United States,France,Argentina,Thailand,Brazil,Canada,India,Colombia,Peru,Chile
1,Vaccine Delivery,147852,168097,55247478,4613924,21752042,32880651,101650,287593,6677944,1084143,693327
2,Vaccine Effectiveness,14332,24029,5039214,57724,67033502,783694,39910,592054,31545,32156,13821
4,Vaccine Incentives,625070,125932,10153452,11977004,17307808,4310652,21575,215893,2461405,378527,306210
5,Vaccine Safety,215,720,3151104,74990,435156,11326,381,470582,1187,157865,2942
6,Vaccine Trial,142700,2177213,7235935,10232772,36301401,34295463,13624,73103,454044,1668872,309461


In [83]:
import plotly.express as px

In [84]:
for loc in locations:
        fig = px.pie(loc_topic_distribution_agg, values=f'{loc}', names='topic',color='topic',
                     title=f'Topic Distribution for {loc}')
        #fig.update_trace(sort=False)
        fig.write_html(f'topic_pie_{loc}.html')

In [85]:
loc_hesitant_agg = loc_hesitant_agg*100
loc_hesitant_agg.columns = locations
loc_hesitant_agg.reset_index(inplace=True)
loc_hesitant_agg=loc_hesitant_agg[loc_hesitant_agg['topic']!='IR']
loc_hesitant_agg=loc_hesitant_agg[loc_hesitant_agg['topic']!='Vaccine General']



In [86]:

for loc in locations:
        fig = px.bar(loc_hesitant_agg, y=f'{loc}', x='topic',color='topic',
                     title=f'Proportion of Hesitant Posts for {loc}')
        fig.update_layout(template='simple_white')
        fig.update_xaxes(title_text = 'Topic')
        fig.update_yaxes(title_text = 'Percentage of Hesitant Posts')

        #fig.update_trace(sort=False)
        fig.write_html(f'topic_bar_{loc}.html')